In [1]:
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader
# import math

# # Placeholder for GeoBLEU and DTW functions
# # In a real-world scenario, you would install the library and import them.
# %pip install git+https://github.com/yahoojapan/geobleu.git
# from geobleu import calc_geobleu_single
# from geobleu import calc_dtw_single
# # The functions are designed to handle (d,t,x,y) tuples for a single user.

In [2]:

# # def calc_geobleu_single(generated, reference):
# #     # This is a dummy implementation based on the repo's logic.
# #     # It would be replaced by the actual function from the installed library.
# #     gen_xy = np.array([(p[2], p[3]) for p in generated])
# #     ref_xy = np.array([(p[2], p[3]) for p in reference])
    
# #     if len(gen_xy) == 0 or len(ref_xy) == 0:
# #         return 0.0
    
# #     # Simulate a basic score calculation for GeoBLEU
# #     distance = np.mean(np.sqrt(np.sum((gen_xy - ref_xy[:len(gen_xy)])**2, axis=1)))
# #     score = max(0, 1.0 - distance / 100)
# #     return score

# # def calc_dtw_single(generated, reference):
# #     # This is a dummy implementation based on the repo's logic.
# #     # It would be replaced by the actual function from the installed library.
# #     gen_xy = np.array([(p[2], p[3]) for p in generated])
# #     ref_xy = np.array([(p[2], p[3]) for p in reference])
    
# #     # Simple DTW cost (not the full algorithm, just for demonstration)
# #     cost = np.sum(np.abs(gen_xy - ref_xy[:len(gen_xy)]))
# #     return cost

# # --- CONFIGURATION: CHANGE THESE PARAMETERS EASILY ---
# FILE_PATHS = {
#     'city_a': '/kaggle/input/humob-data/15313913/city_A_challengedata.csv',
#     # 'city_b': '/kaggle/input/humob-data/15313913/city_B_challengedata.csv',
#     # 'city_c': '/kaggle/input/humob-data/15313913/city_C_challengedata.csv',
#     # 'city_d': '/kaggle/input/humob-data/15313913/city_D_challengedata.csv',
#     # 'city_f' : '/kaggle/input/dummy-data/city_F_challengedata.csv',
# }

# # Training and testing days for the train/test split
# TRAIN_DAYS = range(1, 61) # Days 1 to 60 for all cities
# POST_GAP_DAYS = range(61, 76) # Days 61 to 75
# POST_GAP_TRAIN_FRACTION = 0.5 # 50% of the unmasked data in days 61-75
# BATCH_SIZE = 2
# NUM_EPOCHS = 5
# LSTM_UNITS = 128
# LEARNING_RATE = 0.001

# # --- 1. DATASET AND DATALOADER ---
# class CityDataset(Dataset):
#     def __init__(self, file_path, is_training=True):
#         df = pd.read_csv(file_path)

#         # Filter out masked data (999,999) and unnecessary columns
#         df = df[(df['x'] != 999) & (df['y'] != 999)].copy()
        
#         # Rename columns for clarity as specified in the geobleu repo
#         df.rename(columns={'d': 'day', 't': 'time'}, inplace=True)
        
#         # Feature Engineering
#         df['day_of_week'] = (df['day'] % 7).astype(np.float32)
#         df['is_post_gap'] = (df['day'] > 60).astype(np.float32)
        
#         # Create lagged features
#         for lag in [1, 7]:
#             df[f'x_lag_{lag}'] = df.groupby('uid')['x'].shift(lag)
#             df[f'y_lag_{lag}'] = df.groupby('uid')['y'].shift(lag)
        
#         df.fillna(0, inplace=True)

#         # Split data based on days and the post-gap fraction
#         train_df = df[df['day'].isin(TRAIN_DAYS)]
#         post_gap_df = df[df['day'].isin(POST_GAP_DAYS)]
        
#         uids = post_gap_df['uid'].unique()
#         np.random.seed(42)
#         np.random.shuffle(uids)
        
#         train_uids = uids[:int(len(uids) * POST_GAP_TRAIN_FRACTION)]
        
#         if is_training:
#             post_gap_split = post_gap_df[post_gap_df['uid'].isin(train_uids)]
#             final_df = pd.concat([train_df, post_gap_split])
#         else:
#             test_uids = uids[int(len(uids) * POST_GAP_TRAIN_FRACTION):]
#             final_df = post_gap_df[post_gap_df['uid'].isin(test_uids)]
            
#         # Store for metric calculation later
#         self.raw_data = final_df[['uid', 'day', 'time', 'x', 'y']].values
        
#         # Convert to PyTorch tensors
#         feature_cols = [
#             'day', 'time', 'day_of_week', 'is_post_gap',
#             'x_lag_1', 'y_lag_1', 'x_lag_7', 'y_lag_7'
#         ]
#         self.features = torch.tensor(final_df[feature_cols].values, dtype=torch.float32)
#         self.labels = torch.tensor(final_df[['x', 'y']].values, dtype=torch.float32)
        
#     def __len__(self):
#         return len(self.features)
        
#     def __getitem__(self, idx):
#         return self.features[idx], self.labels[idx], self.raw_data[idx]

# # --- 2. MODEL ARCHITECTURE ---
# class LSTMModel(nn.Module):
#     def __init__(self, input_size, hidden_size=LSTM_UNITS, num_layers=1, output_size=2):
#         super(LSTMModel, self).__init__()
#         self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
#         self.fc = nn.Linear(hidden_size, output_size)

#     def forward(self, x):
#         x = x.unsqueeze(1) 
#         h_lstm, _ = self.lstm(x)
#         h_lstm = h_lstm[:, -1, :]
#         out = self.fc(h_lstm)
#         return out

# # --- 3. METRICS CALCULATION ---
# def calculate_metrics(y_true, y_pred, raw_data):
#     rmse = np.sqrt(np.mean((y_true - y_pred)**2))

#     # Convert true and predicted values to the (day, time, x, y) format
#     # The geobleu repo expects this format for single user evaluation.
#     true_trajectories = {}
#     pred_trajectories = {}
    
#     for i in range(len(raw_data)):
#         uid, day, time = raw_data[i][:3]
#         true_traj = (day, time, y_true[i][0], y_true[i][1])
#         pred_traj = (day, time, y_pred[i][0], y_pred[i][1])
        
#         if uid not in true_trajectories:
#             true_trajectories[uid] = []
#             pred_trajectories[uid] = []
#         true_trajectories[uid].append(true_traj)
#         pred_trajectories[uid].append(pred_traj)

#     # Calculate GeoBLEU and DTW per user and average
#     geobleu_scores = []
#     dtw_scores = []
#     for uid in true_trajectories:
#         geobleu_scores.append(calc_geobleu_single(pred_trajectories[uid], true_trajectories[uid]))
#         dtw_scores.append(calc_dtw_single(pred_trajectories[uid], true_trajectories[uid]))

#     avg_geobleu = np.mean(geobleu_scores)
#     avg_dtw = np.mean(dtw_scores)

#     print(f"Metrics:")
#     print(f"  RMSE Score: {rmse:.4f}")
#     print(f"  GeoBLEU Score: {avg_geobleu:.4f}")
#     print(f"  DTW Score: {avg_dtw:.4f}")


# # --- 4. MAIN EXECUTION ---
# if __name__ == "__main__":
#     # Create dummy data for demonstration
#     def create_dummy_data(filename, num_rows, discontinuity_day=None):
#         df = pd.DataFrame({
#             'uid': np.random.randint(0, 1000, size=num_rows),
#             'x': np.random.rand(num_rows) * 100,
#             'y': np.random.rand(num_rows) * 100,
#             'd': np.arange(1, num_rows + 1) % 76 + 1,
#             't': np.random.randint(1, 97, size=num_rows)
#         })
#         if discontinuity_day:
#             df.loc[df['d'] > discontinuity_day, ['x', 'y']] += 1.0
#         df.loc[df['d'].isin(range(76, 100)), ['x', 'y']] = 999.0
#         df.to_csv(filename, index=False)
    
    
#     for city_name, file_path in FILE_PATHS.items():
#         print(f"\n--- Training and evaluating for {city_name} ---")
        
#         # Load datasets
#         train_dataset = CityDataset(file_path, is_training=True)
#         test_dataset = CityDataset(file_path, is_training=False)
        
#         if len(train_dataset) == 0 or len(test_dataset) == 0:
#             print("  No data available for training or testing. Skipping.")
#             continue
            
#         # Create DataLoaders
#         train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
#         test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
        
#         # Initialize model, loss function, and optimizer
#         input_size = train_dataset.features.shape[1]
#         model = LSTMModel(input_size)
#         criterion = nn.MSELoss()
#         optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
        
#         # Training loop
#         model.train()
#         for epoch in range(NUM_EPOCHS):
#             for features, labels, _ in train_loader:
#                 outputs = model(features)
#                 loss = criterion(outputs, labels)
                
#                 optimizer.zero_grad()
#                 loss.backward()
#                 optimizer.step()
            
#             print(f"  Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {loss.item():.4f}")
        
#         # Evaluation loop
#         model.eval()
#         with torch.no_grad():
#             y_true_list = []
#             y_pred_list = []
#             raw_data_list = []
#             for features, labels, raw_data in test_loader:
#                 outputs = model(features)
#                 y_true_list.append(labels.numpy())
#                 y_pred_list.append(outputs.numpy())
#                 raw_data_list.append(raw_data.numpy())
            
#             y_true = np.concatenate(y_true_list)
#             y_pred = np.concatenate(y_pred_list)
#             raw_data = np.concatenate(raw_data_list)
            
#             calculate_metrics(y_true, y_pred, raw_data)

In [3]:
# # Safe Per-City LSTM baseline (Colab / Kaggle)
# # - Primary metric: GeoBLEU (use official library)
# # - Training split: days 1-60 + fraction of unmasked days 61-75 -> train
# #                  remaining unmasked days 61-75 -> test
# #
# # IMPORTANT: install geobleu in the notebook environment before running:
# %pip install git+https://github.com/yahoojapan/geobleu.git

# import os
# import math
# import gc
# import random
# from collections import defaultdict
# from typing import List, Tuple, Dict

# import numpy as np
# import pandas as pd

# import torch
# import torch.nn as nn
# from torch.utils.data import DataLoader, Dataset

# # -------- CONFIG (tune conservatively for Colab/Kaggle) ----------
# FILE_PATHS = {
#     # 'city_a': '/kaggle/input/humob-data/15313913/city_A_challengedata.csv',
#     'city_b': '/kaggle/input/humob-data/15313913/city_B_challengedata.csv',
#     'city_c': '/kaggle/input/humob-data/15313913/city_C_challengedata.csv',
#     'city_d': '/kaggle/input/humob-data/15313913/city_D_challengedata.csv',
# }

# TRAIN_DAYS = set(range(1,61))
# POST_GAP_DAYS = set(range(61,76))
# POST_GAP_TRAIN_FRACTION = 0.5

# CHUNKSIZE = 400_000      # reduce if RAM pressure; increase for speed if RAM allows
# SEQ_LEN = 8              # short sequence length (safe)
# BATCH_SIZE = 64         # tune down to 64/32 if OOM
# NUM_EPOCHS = 4
# LR = 1e-3
# HIDDEN = 128
# NUM_LAYERS = 1

# FP16 = True              # use mixed precision when GPU available
# GRAD_ACCUM_STEPS = 2     # set >1 to simulate larger batch without OOM

# SEED = 42
# random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

# DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# print("Device:", DEVICE)

# # GeoBLEU import (official)
# try:
#     from geobleu import calc_geobleu_single, calc_dtw_single
# except Exception:
#     # If not installed, raise clear message
#     raise RuntimeError("Please install GeoBLEU: %pip install git+https://github.com/yahoojapan/geobleu.git")

# # ------------------ Utilities ------------------
# def add_features(df: pd.DataFrame) -> pd.DataFrame:
#     """Basic feature engineering and mask removal. Drop masked rows here."""
#     df = df.rename(columns={'d':'day','t':'time'})
#     # drop masked rows (we can't train on masked)
#     df = df[(df['x'] != 999) & (df['y'] != 999)].copy()
#     if df.empty:
#         return df
#     # cyclical time features: assume time slots are integer slots; use local max  (safe)
#     max_t = max(1, int(df['time'].max()))
#     df['time_sin'] = np.sin(2*np.pi*(df['time']-1)/max_t).astype(np.float32)
#     df['time_cos'] = np.cos(2*np.pi*(df['time']-1)/max_t).astype(np.float32)
#     df['day_of_week'] = (df['day'] % 7).astype(np.int8)
#     df['dow_sin'] = np.sin(2*np.pi*(df['day_of_week'])/7).astype(np.float32)
#     df['dow_cos'] = np.cos(2*np.pi*(df['day_of_week'])/7).astype(np.float32)
#     df['is_post_gap'] = (df['day'] > 60).astype(np.float32)
#     return df

# def uid_postgap_split_iter(csv_path: str, sample_chunks: int = 5) -> Tuple[set,set]:
#     """Get train_uids/test_uids among unmasked rows in days 61-75.
#     streaming limited by sample_chunks (fast heuristic)."""
#     uids = []
#     chunks_seen = 0
#     for chunk in pd.read_csv(csv_path, usecols=['uid','x','y','d','t'], chunksize=CHUNKSIZE):
#         chunk = chunk.rename(columns={'d':'day','t':'time'})
#         chunk = chunk[(chunk['day'].isin(POST_GAP_DAYS)) & (chunk['x'] != 999) & (chunk['y'] != 999)]
#         if len(chunk):
#             uids.append(chunk['uid'].unique())
#         chunks_seen += 1
#         if chunks_seen >= sample_chunks:
#             break
#     if not uids:
#         return set(), set()
#     all_uids = np.concatenate(uids)
#     all_uids = np.unique(all_uids)
#     rng = np.random.default_rng(SEED)
#     rng.shuffle(all_uids)
#     split = int(len(all_uids) * POST_GAP_TRAIN_FRACTION)
#     train_uids = set(all_uids[:split])
#     test_uids = set(all_uids[split:])
#     return train_uids, test_uids

# def build_chunk_sequences(chunk: pd.DataFrame, seq_len: int, allowed_uids: set=None, allowed_days: set=None):
#     """Return list of (X_seq_array, y_next_array, meta) from this chunk.
#     Sequences are built per-UID using only rows present in chunk (safe & memory-light).
#     """
#     if chunk.empty:
#         return []
#     cols = ['x','y','time_sin','time_cos','dow_sin','dow_cos','is_post_gap']
#     # If allowed_days filter provided
#     if allowed_days is not None:
#         chunk = chunk[chunk['day'].isin(allowed_days)]
#     if allowed_uids is not None:
#         chunk = chunk[chunk['uid'].isin(allowed_uids)]
#     if chunk.empty:
#         return []
#     # sort for safety
#     chunk = chunk.sort_values(['uid','day','time'])
#     seqs = []
#     for uid, g in chunk.groupby('uid'):
#         arr = g[cols].values.astype(np.float32)
#         meta_arr = g[['day','time','x','y']].values
#         L = len(arr)
#         if L <= seq_len:
#             continue
#         # sliding windows inside chunk only
#         for i in range(L - seq_len):
#             X = arr[i:i+seq_len]          # [seq_len, feat]
#             y_row = meta_arr[i+seq_len]   # this row has day,time,x,y for label
#             y = np.array([y_row[2], y_row[3]], dtype=np.float32)
#             meta = (int(uid), int(y_row[0]), int(y_row[1]))
#             seqs.append((X, y, meta))
#     return seqs

# # ------------------ Model ------------------
# class LSTMModel(nn.Module):
#     def __init__(self, input_dim=7, hidden=HIDDEN, num_layers=NUM_LAYERS):
#         super().__init__()
#         self.lstm = nn.LSTM(input_dim, hidden, num_layers, batch_first=True)
#         self.fc = nn.Linear(hidden, 2)
#     def forward(self, x):
#         # x: [B, seq_len, feat]
#         out, _ = self.lstm(x)
#         last = out[:, -1, :]
#         return self.fc(last)

# # ------------------ Training/Evaluation helpers ------------------
# def train_on_chunk(model, optimizer, scaler, chunk_seqs, device, criterion, accum_steps=1):
#     """Train model on a *list* of sequences from a single chunk, then drop them."""
#     if not chunk_seqs:
#         return None
#     model.train()
#     running_loss = 0.0
#     cnt = 0
#     batch_X, batch_y = [], []
#     for (X, y, _) in chunk_seqs:
#         batch_X.append(X)
#         batch_y.append(y)
#         if len(batch_X) >= BATCH_SIZE:
#             Xb = torch.tensor(np.stack(batch_X), dtype=torch.float32, device=device)
#             yb = torch.tensor(np.stack(batch_y), dtype=torch.float32, device=device)
#             with torch.amp.autocast('cuda', enabled=FP16 and device.startswith('cuda')):
#                 pred = model(Xb)
#                 loss = criterion(pred, yb) / accum_steps
#             scaler.scale(loss).backward()
#             if (cnt+1) % accum_steps == 0:
#                 scaler.unscale_(optimizer)
#                 torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#                 scaler.step(optimizer)
#                 scaler.update()
#                 optimizer.zero_grad(set_to_none=True)
#             running_loss += float(loss.item()) * accum_steps
#             batch_X, batch_y = [], []
#             cnt += 1
#     # final partial batch
#     if batch_X:
#         Xb = torch.tensor(np.stack(batch_X), dtype=torch.float32, device=device)
#         yb = torch.tensor(np.stack(batch_y), dtype=torch.float32, device=device)
#         with torch.amp.autocast('cuda', enabled=FP16 and device.startswith('cuda')):
#             pred = model(Xb)
#             loss = criterion(pred, yb) / accum_steps
#         scaler.scale(loss).backward()
#         scaler.unscale_(optimizer)
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#         scaler.step(optimizer)
#         scaler.update()
#         optimizer.zero_grad(set_to_none=True)
#         running_loss += float(loss.item()) * accum_steps
#     return running_loss

# def evaluate_city(model, csv_path, test_uids, device):
#     """Stream through CSV, build sequences for test_uids and produce predictions per uid,
#     then compute GeoBLEU & DTW averaged over uids."""
#     model.eval()
#     per_uid_true = defaultdict(list)
#     per_uid_pred = defaultdict(list)
#     # stream through chunks and build sequences for allowed uids and days=POST_GAP_DAYS
#     for chunk in pd.read_csv(csv_path, usecols=['uid','x','y','d','t'], chunksize=CHUNKSIZE):
#         chunk = chunk.rename(columns={'d':'day','t':'time'})
#         chunk = add_features(chunk)
#         if chunk.empty:
#             continue
#         # limit to days 61-75 and uids in test set
#         chunk = chunk[(chunk['day'].isin(POST_GAP_DAYS)) & (chunk['uid'].isin(test_uids))]
#         if chunk.empty:
#             continue
#         seqs = build_chunk_sequences(chunk, SEQ_LEN, allowed_uids=test_uids, allowed_days=POST_GAP_DAYS)
#         # run model on sequences and append per-uid lists
#         if not seqs:
#             continue
#         # batch predict
#         for i in range(0, len(seqs), BATCH_SIZE):
#             batch = seqs[i:i+BATCH_SIZE]
#             Xs = np.stack([s[0] for s in batch])
#             metas = [s[2] for s in batch]
#             Xb = torch.tensor(Xs, dtype=torch.float32, device=device)
#             with torch.no_grad(), torch.cuda.amp.autocast(enabled=FP16 and device.startswith('cuda')):
#                 preds = model(Xb).cpu().numpy()
#             for (uid, day, time), p, s in zip(metas, preds, [b[1] for b in batch]):
#                 per_uid_pred[uid].append((int(day), int(time), float(p[0]), float(p[1])))
#                 per_uid_true[uid].append((int(day), int(time), float(s[0]), float(s[1])))
#         # free memory
#         del seqs, chunk
#         gc.collect()

#     # compute metrics per-uid (use calc_geobleu_single, calc_dtw_single)
#     geobleu_scores = []
#     dtw_scores = []
#     for uid in per_uid_true:
#         # ensure both lists are sorted
#         true_seq = sorted(per_uid_true[uid], key=lambda x:(x[0],x[1]))
#         pred_seq = sorted(per_uid_pred.get(uid, []), key=lambda x:(x[0],x[1]))
#         if len(true_seq) == 0 or len(pred_seq) == 0:
#             continue
#         try:
#             g = calc_geobleu_single(pred_seq, true_seq)
#             d = calc_dtw_single(pred_seq, true_seq)
#         except Exception:
#             # In very rare cases if calc_geobleu_single expects different format, fallback simple measure
#             # But you installed official library, so this should not trigger.
#             g = 0.0
#             d = float('inf')
#         geobleu_scores.append(g)
#         dtw_scores.append(d)
#     avg_geo = float(np.mean(geobleu_scores)) if geobleu_scores else 0.0
#     avg_dtw = float(np.mean(dtw_scores)) if dtw_scores else float('inf')
#     return avg_geo, avg_dtw, len(per_uid_true)

# # ------------------ Main per-city loop ------------------
# def train_city(csv_path, city_name, device=DEVICE):
#     print(f"\n=== City: {city_name} ===")
#     train_uids, test_uids = uid_postgap_split_iter(csv_path, sample_chunks=8)
#     if not train_uids and not test_uids:
#         print("No unmasked post-gap uids found (check CSV). Exiting city.")
#         return
#     print(f"Train UIDs (sampled): {len(train_uids)} ; Test UIDs (sampled): {len(test_uids)}")

#     model = LSTMModel(input_dim=7, hidden=HIDDEN, num_layers=NUM_LAYERS).to(device)
#     optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
#     criterion = nn.SmoothL1Loss()
#     scaler = torch.cuda.amp.GradScaler(enabled=FP16 and device.startswith('cuda'))

#     best_geo = -1.0
#     early = 0

#     for epoch in range(1, NUM_EPOCHS+1):
#         print(f"Epoch {epoch}/{NUM_EPOCHS}")
#         total_loss = 0.0
#         chunks = 0
#         # stream chunks and train on sequences from chunk (train days: 1-60 + postgap train_uids)
#         for chunk in pd.read_csv(csv_path, usecols=['uid','x','y','d','t'], chunksize=CHUNKSIZE):
#             chunk = chunk.rename(columns={'d':'day','t':'time'})
#             chunk = add_features(chunk)
#             if chunk.empty:
#                 continue
#             # limit chunk to training rows: days 1-60 OR (days 61-75 AND uid in train_uids)
#             mask = (chunk['day'].isin(TRAIN_DAYS)) | ((chunk['day'].isin(POST_GAP_DAYS)) & (chunk['uid'].isin(train_uids)))
#             chunk_train = chunk[mask]
#             if chunk_train.empty:
#                 continue
#             seqs = build_chunk_sequences(chunk_train, SEQ_LEN, allowed_uids=None, allowed_days=None)
#             if not seqs:
#                 continue
#             loss = train_on_chunk(model, optimizer, scaler, seqs, device, criterion, accum_steps=GRAD_ACCUM_STEPS)
#             if loss is not None:
#                 total_loss += loss
#             chunks += 1
#             # keep memory low
#             del seqs, chunk_train
#             gc.collect()
#         avg_loss = total_loss / max(1, chunks)
#         print(f"  Avg chunk loss: {avg_loss:.5f} over {chunks} chunks")

#         # validation after each epoch
#         geo, dtw, n_uids = evaluate_city(model, csv_path, test_uids, device)
#         print(f"  Val GeoBLEU: {geo:.6f} | Val DTW: {dtw:.4f} | UIDs evaluated: {n_uids}")

#         # early save if improves
#         if geo > best_geo:
#             best_geo = geo
#             early = 0
#             # save model
#             ckpt = f"{city_name}_best.pt"
#             torch.save({'model_state': model.state_dict(), 'optimizer': optimizer.state_dict()}, ckpt)
#             print("  Saved best checkpoint:", ckpt)
#         else:
#             early += 1
#             if early >= 3:
#                 print("  Early stopping triggered.")
#                 break

#     print(f"Finished training {city_name}. Best GeoBLEU: {best_geo:.6f}")

#     # final evaluation (full) - you can re-run evaluate_city for full final metrics if desired
#     return model

# # ------------------ Run for each configured city ------------------
# if __name__ == "__main__":
#     for city, path in FILE_PATHS.items():
#         if not os.path.exists(path):
#             print(f"[WARN] Missing {path}, skipping {city}")
#             continue
#         train_city(path, city, DEVICE)

#     # -----------------------------
#     # (COMMENTED) Masked data prediction for final submission:
#     # -----------------------------
#     # To create predictions for masked rows (x=999 & y=999), follow these steps:
#     # 1) Load the saved best model checkpoint for city.
#     # 2) Stream CSV in chunks, but this time filter chunk to rows with (x==999 and y==999)
#     # 3) For each masked UID/day/time, construct the input sequence from latest available previous unmasked rows
#     #    (you may need a small per-uid cache tracking last SEQ_LEN unmasked points).
#     # 4) Run model on the sequence and write predicted (x,y) into submission CSV in required format.
#     #
#     # Pseudocode:
#     #   cache = defaultdict(list)   # uid -> recent unmasked (x,y,time,dow,is_post_gap...)
#     #   for chunk in pd.read_csv(..., chunksize=CHUNKSIZE):
#     #       for row in chunk rows:
#     #           if row unmasked: update cache[uid]
#     #           else:  # masked -> predict if cache[uid] has SEQ_LEN entries
#     #               X = build_features_from_cache(cache[uid][-SEQ_LEN:])
#     #               pred = model(torch.tensor(X).unsqueeze(0).to(device))
#     #               write pred to submission
#     #
#     # Note: predicting masked rows requires careful handling of time slots and continuity.


In [4]:
# Safe Per-City LSTM baseline (Colab / Kaggle)
# - Primary metric: GeoBLEU (use official library)
# - Training split: days 1-60 + fraction of unmasked days 61-75 -> train
#                  remaining unmasked days 61-75 -> test
#
# IMPORTANT: install geobleu in the notebook environment before running:
%pip install git+https://github.com/yahoojapan/geobleu.git

import os
import math
import gc
import random
from collections import defaultdict
from typing import Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

# -------- CONFIG (tune conservatively for Colab/Kaggle) ----------
FILE_PATHS = {
    # 'city_a': '/kaggle/input/humob-data/15313913/city_A_challengedata.csv',
    # 'city_b': '/kaggle/input/humob-data/15313913/city_B_challengedata.csv',
    # 'city_c': '/kaggle/input/humob-data/15313913/city_C_challengedata.csv',
    'city_d': '/kaggle/input/humob-data/15313913/city_D_challengedata.csv',
}

TRAIN_DAYS = set(range(1, 61))
POST_GAP_DAYS = set(range(61, 76))
POST_GAP_TRAIN_FRACTION = 0.5

CHUNKSIZE = 400_000      # reduce if RAM pressure; increase for speed if RAM allows
SEQ_LEN = 8              # short sequence length (safe)
BATCH_SIZE = 64          # tune down to 64/32 if OOM
NUM_EPOCHS = 10           # modest epochs for large-scale data; tune later if needed
LR = 1e-3
HIDDEN = 128
NUM_LAYERS = 1

FP16 = True              # use mixed precision when GPU available
GRAD_ACCUM_STEPS = 2     # set >1 to simulate larger batch without OOM

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", DEVICE, "| CUDA available:", torch.cuda.is_available())

# GeoBLEU import (official)
try:
    from geobleu import calc_geobleu_single, calc_dtw_single
except Exception as e:
    raise RuntimeError("Please install GeoBLEU: %pip install git+https://github.com/yahoojapan/geobleu.git") from e

# ------------------ Utilities ------------------
def add_features(df: pd.DataFrame) -> pd.DataFrame:
    """Basic feature engineering and mask removal. Drop masked rows here."""
    df = df.rename(columns={'d': 'day', 't': 'time'})
    df = df[(df['x'] != 999) & (df['y'] != 999)].copy()
    if df.empty:
        return df
    max_t = max(1, int(df['time'].max()))
    df['time_sin'] = np.sin(2 * np.pi * (df['time'] - 1) / max_t).astype(np.float32)
    df['time_cos'] = np.cos(2 * np.pi * (df['time'] - 1) / max_t).astype(np.float32)
    df['day_of_week'] = (df['day'] % 7).astype(np.int8)
    df['dow_sin'] = np.sin(2 * np.pi * (df['day_of_week']) / 7).astype(np.float32)
    df['dow_cos'] = np.cos(2 * np.pi * (df['day_of_week']) / 7).astype(np.float32)
    df['is_post_gap'] = (df['day'] > 60).astype(np.float32)
    return df

def uid_postgap_split_iter(csv_path: str, sample_chunks: int = 8) -> Tuple[set, set]:
    """Get train_uids/test_uids among unmasked rows in days 61-75.
    Streaming limited by sample_chunks (fast heuristic)."""
    uids = []
    chunks_seen = 0
    for chunk in pd.read_csv(csv_path, usecols=['uid', 'x', 'y', 'd', 't'], chunksize=CHUNKSIZE):
        chunk = chunk.rename(columns={'d': 'day', 't': 'time'})
        chunk = chunk[(chunk['day'].isin(POST_GAP_DAYS)) & (chunk['x'] != 999) & (chunk['y'] != 999)]
        if len(chunk):
            uids.append(chunk['uid'].unique())
        chunks_seen += 1
        if chunks_seen >= sample_chunks:
            break
    if not uids:
        return set(), set()
    all_uids = np.unique(np.concatenate(uids))
    rng = np.random.default_rng(SEED)
    rng.shuffle(all_uids)
    split = int(len(all_uids) * POST_GAP_TRAIN_FRACTION)
    train_uids = set(all_uids[:split])
    test_uids = set(all_uids[split:])
    return train_uids, test_uids

def build_chunk_sequences(chunk: pd.DataFrame, seq_len: int, allowed_uids: set = None, allowed_days: set = None):
    """Return list of (X_seq_array, y_next_array, meta) from this chunk.
    Sequences are built per-UID using only rows present in chunk (safe & memory-light).
    """
    if chunk.empty:
        return []
    cols = ['x', 'y', 'time_sin', 'time_cos', 'dow_sin', 'dow_cos', 'is_post_gap']
    if allowed_days is not None:
        chunk = chunk[chunk['day'].isin(allowed_days)]
    if allowed_uids is not None:
        chunk = chunk[chunk['uid'].isin(allowed_uids)]
    if chunk.empty:
        return []
    chunk = chunk.sort_values(['uid', 'day', 'time'])
    seqs = []
    for uid, g in chunk.groupby('uid'):
        arr = g[cols].values.astype(np.float32)
        meta_arr = g[['day', 'time', 'x', 'y']].values
        L = len(arr)
        if L <= seq_len:
            continue
        for i in range(L - seq_len):
            X = arr[i:i + seq_len]
            y_row = meta_arr[i + seq_len]
            y = np.array([y_row[2], y_row[3]], dtype=np.float32)
            meta = (int(uid), int(y_row[0]), int(y_row[1]))
            seqs.append((X, y, meta))
    return seqs

# ------------------ Model ------------------
class LSTMModel(nn.Module):
    def __init__(self, input_dim: int = 7, hidden: int = HIDDEN, num_layers: int = NUM_LAYERS):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden, 2)

    def forward(self, x):
        # x: [B, seq_len, feat]
        out, _ = self.lstm(x)
        last = out[:, -1, :]
        return self.fc(last)

class ComboLoss(nn.Module):
    """SmoothL1 + Cosine similarity (directional) for spatial stability."""
    def __init__(self, alpha: float = 0.8, beta: float = 0.2, eps: float = 1e-6):
        super().__init__()
        self.l1 = nn.SmoothL1Loss()
        self.alpha = alpha
        self.beta = beta
        self.eps = eps

    def forward(self, pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        l_reg = self.l1(pred, target)
        p = pred + self.eps
        t = target + self.eps
        cos = (p * t).sum(dim=1) / (p.norm(dim=1) * t.norm(dim=1) + self.eps)
        l_cos = 1 - cos.mean()
        return self.alpha * l_reg + self.beta * l_cos

# ------------------ Training/Evaluation helpers ------------------
def train_on_chunk(model, optimizer, scaler, chunk_seqs, device, criterion, accum_steps=1):
    """Train model on a *list* of sequences from a single chunk, then drop them."""
    if not chunk_seqs:
        return None
    model.train()
    running_loss = 0.0
    cnt = 0
    batch_X, batch_y = [], []
    for (X, y, _) in chunk_seqs:
        batch_X.append(X)
        batch_y.append(y)
        if len(batch_X) >= BATCH_SIZE:
            Xb = torch.tensor(np.stack(batch_X), dtype=torch.float32, device=device)
            yb = torch.tensor(np.stack(batch_y), dtype=torch.float32, device=device)
            with torch.amp.autocast(device_type='cuda', enabled=FP16 and device.type == 'cuda'):
                pred = model(Xb)
                loss = criterion(pred, yb) / accum_steps
            scaler.scale(loss).backward()
            if (cnt + 1) % accum_steps == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad(set_to_none=True)
            running_loss += float(loss.item()) * accum_steps
            batch_X, batch_y = [], []
            cnt += 1
    # final partial batch
    if batch_X:
        Xb = torch.tensor(np.stack(batch_X), dtype=torch.float32, device=device)
        yb = torch.tensor(np.stack(batch_y), dtype=torch.float32, device=device)
        with torch.amp.autocast(device_type='cuda', enabled=FP16 and device.type == 'cuda'):
            pred = model(Xb)
            loss = criterion(pred, yb) / accum_steps
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad(set_to_none=True)
        running_loss += float(loss.item()) * accum_steps
    return running_loss

def evaluate_city(model, csv_path, test_uids, device):
    """Stream through CSV, build sequences for test_uids and produce predictions per uid,
    then compute GeoBLEU & DTW averaged over uids."""
    model.eval()
    per_uid_true = defaultdict(list)
    per_uid_pred = defaultdict(list)
    for chunk in pd.read_csv(csv_path, usecols=['uid', 'x', 'y', 'd', 't'], chunksize=CHUNKSIZE):
        chunk = chunk.rename(columns={'d': 'day', 't': 'time'})
        chunk = add_features(chunk)
        if chunk.empty:
            continue
        chunk = chunk[(chunk['day'].isin(POST_GAP_DAYS)) & (chunk['uid'].isin(test_uids))]
        if chunk.empty:
            continue
        seqs = build_chunk_sequences(chunk, SEQ_LEN, allowed_uids=test_uids, allowed_days=POST_GAP_DAYS)
        if not seqs:
            continue
        for i in range(0, len(seqs), BATCH_SIZE):
            batch = seqs[i:i + BATCH_SIZE]
            Xs = np.stack([s[0] for s in batch])
            metas = [s[2] for s in batch]
            Xb = torch.tensor(Xs, dtype=torch.float32, device=device)
            with torch.no_grad(), torch.amp.autocast(device_type='cuda', enabled=FP16 and device.type == 'cuda'):
                preds = model(Xb).cpu().numpy()
            for (uid, day, time), p, s in zip(metas, preds, [b[1] for b in batch]):
                per_uid_pred[uid].append((int(day), int(time), float(p[0]), float(p[1])))
                per_uid_true[uid].append((int(day), int(time), float(s[0]), float(s[1])))
        del seqs, chunk
        gc.collect()

    geobleu_scores = []
    dtw_scores = []
    for uid in per_uid_true:
        true_seq = sorted(per_uid_true[uid], key=lambda x: (x[0], x[1]))
        pred_seq = sorted(per_uid_pred.get(uid, []), key=lambda x: (x[0], x[1]))
        if len(true_seq) == 0 or len(pred_seq) == 0:
            continue
        try:
            g = calc_geobleu_single(pred_seq, true_seq)
            d = calc_dtw_single(pred_seq, true_seq)
        except Exception:
            g = 0.0
            d = float('inf')
        geobleu_scores.append(g)
        dtw_scores.append(d)
    avg_geo = float(np.mean(geobleu_scores)) if geobleu_scores else 0.0
    avg_dtw = float(np.mean(dtw_scores)) if dtw_scores else float('inf')
    return avg_geo, avg_dtw, len(per_uid_true)

# ------------------ Helpers: plotting & cleanup ------------------
def plot_losses(city_name: str, epoch_losses: list):
    try:
        plt.figure()
        plt.plot(range(1, len(epoch_losses) + 1), epoch_losses, marker='o')
        plt.xlabel('Epoch')
        plt.ylabel('Average Chunk Loss')
        plt.title(f'Loss Curve - {city_name}')
        plt.grid(True)
        plt.tight_layout()
        png = f"{city_name}_loss_curve.png"
        plt.savefig(png)
        print(f"Saved loss curve: {png}")
        plt.close()
    except Exception as e:
        print("[WARN] Could not plot loss curve:", e)

def cleanup_gpu_and_exit(force_terminate: bool = False):
    """Best-effort cleanup. NOTE: You cannot programmatically *disable* Kaggle's
    accelerator from code; that setting is controlled by the 'Save Version' pane.
    This frees CUDA memory and optionally terminates the process."""
    try:
        if torch.cuda.is_available():
            torch.cuda.synchronize()
            torch.cuda.empty_cache()
            # torch.cuda.ipc_collect may not exist on older torch; guard it
            if hasattr(torch.cuda, 'ipc_collect'):
                torch.cuda.ipc_collect()
        gc.collect()
    except Exception as e:
        print("Cleanup warning:", e)
    if force_terminate:
        # Hard exit - ends the process immediately. Kaggle will free the runtime after job end.
        os._exit(0)

# ------------------ Main per-city loop ------------------
def train_city(csv_path, city_name, device=DEVICE):
    print(f"=== City: {city_name} ===")
    train_uids, test_uids = uid_postgap_split_iter(csv_path, sample_chunks=8)
    if not train_uids and not test_uids:
        print("No unmasked post-gap uids found (check CSV). Exiting city.")
        return
    print(f"Train UIDs (sampled): {len(train_uids)} ; Test UIDs (sampled): {len(test_uids)}")

    model = LSTMModel(input_dim=7, hidden=HIDDEN, num_layers=NUM_LAYERS).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    criterion = ComboLoss(alpha=0.8, beta=0.2)
    scaler = torch.amp.GradScaler(enabled=FP16 and device.type == 'cuda')

    epoch_losses = []

    for epoch in range(1, NUM_EPOCHS + 1):
        print(f"Epoch {epoch}/{NUM_EPOCHS}")
        total_loss = 0.0
        chunks = 0
        for chunk in pd.read_csv(csv_path, usecols=['uid', 'x', 'y', 'd', 't'], chunksize=CHUNKSIZE):
            chunk = chunk.rename(columns={'d': 'day', 't': 'time'})
            chunk = add_features(chunk)
            if chunk.empty:
                continue
            mask = (chunk['day'].isin(TRAIN_DAYS)) | ((chunk['day'].isin(POST_GAP_DAYS)) & (chunk['uid'].isin(train_uids)))
            chunk_train = chunk[mask]
            if chunk_train.empty:
                continue
            seqs = build_chunk_sequences(chunk_train, SEQ_LEN, allowed_uids=None, allowed_days=None)
            if not seqs:
                continue
            loss = train_on_chunk(model, optimizer, scaler, seqs, device, criterion, accum_steps=GRAD_ACCUM_STEPS)
            if loss is not None:
                total_loss += loss
            chunks += 1
            del seqs, chunk_train
            gc.collect()
        avg_loss = total_loss / max(1, chunks)
        epoch_losses.append(avg_loss)
        print(f"  Avg chunk loss: {avg_loss:.5f} over {chunks} chunks")

    # Save final checkpoint after training
    ckpt = f"{city_name}_final.pt"
    torch.save({'model_state': model.state_dict(), 'optimizer': optimizer.state_dict()}, ckpt)
    print("  Saved final checkpoint:", ckpt)

    # Plot & save loss curve
    plot_losses(city_name, epoch_losses)

    # Final evaluation (single pass) on test_uids
    geo, dtw, n_uids = evaluate_city(model, csv_path, test_uids, device)
    print(f"Final GeoBLEU: {geo:.6f} | Final DTW: {dtw:.4f} | UIDs evaluated: {n_uids}")

    return model

# ------------------ Run for each configured city ------------------
if __name__ == "__main__":
    try:
        for city, path in FILE_PATHS.items():
            if not os.path.exists(path):
                print(f"[WARN] Missing {path}, skipping {city}")
                continue
            train_city(path, city, DEVICE)
    finally:
        # Best-effort cleanup; to force process termination uncomment the next line (careful).
        cleanup_gpu_and_exit(force_terminate=False)
        # If you want the script to hard-exit immediately (releases process), use:
        # cleanup_gpu_and_exit(force_terminate=True)


  Cloning https://github.com/yahoojapan/geobleu.git to /tmp/pip-req-build-niz35_h2
  Running command git clone --filter=blob:none --quiet https://github.com/yahoojapan/geobleu.git /tmp/pip-req-build-niz35_h2
  Resolved https://github.com/yahoojapan/geobleu.git to commit d9efb0fbc97d75b85f273a5db5f4da2215b0f715
  Preparing metadata (setup.py) ... done
  Created wheel for geobleu: filename=geobleu-0.5-py3-none-any.whl size=5442 sha256=33374acc37a70bc35e55bff04766444d949349421351a5d340fa230571689b47
  Stored in directory: /tmp/pip-ephem-wheel-cache-myywlb8f/wheels/97/c3/ec/b654420f0a3507ff7c5234dc42ee565578300e789e6705ec17
Successfully built geobleu
Note: you may need to restart the kernel to use updated packages.
Using device: cuda | CUDA available: True
=== City: city_d ===
Train UIDs (sampled): 2449 ; Test UIDs (sampled): 2450
Epoch 1/10
  Avg chunk loss: 30180.42853 over 30 chunks
Epoch 2/10
  Avg chunk loss: 23068.45856 over 30 chunks
Epoch 3/10
  Avg chunk loss: 22574.74936 over 30 